In [2]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from six.moves import range
from sklearn.metrics import confusion_matrix
import tensorflow as tf
print(tf.__version__)

%matplotlib inline

1.3.0


In [1]:
train_folders = ['../unit-1/notMNIST_large/A', '../unit-1/notMNIST_large/B', '../unit-1/notMNIST_large/C', 
                 '../unit-1/notMNIST_large/D', '../unit-1/notMNIST_large/E', '../unit-1/notMNIST_large/F', 
                 '../unit-1/notMNIST_large/G', '../unit-1/notMNIST_large/H', '../unit-1/notMNIST_large/I', '../unit-1/notMNIST_large/J']
test_folders = ['../unit-1/notMNIST_small/A', '../unit-1/notMNIST_small/B', '../unit-1/notMNIST_small/C', 
                '../unit-1/notMNIST_small/D', '../unit-1/notMNIST_small/E', '../unit-1/notMNIST_small/F', 
                '../unit-1/notMNIST_small/G', '../unit-1/notMNIST_small/H', '../unit-1/notMNIST_small/I', '../unit-1/notMNIST_small/J']

In [3]:
train_datasets = []
for folder in train_folders:
  set_filename = folder + '.pickle'
  train_datasets.append(set_filename)
    
test_datasets = []
for folder in test_folders:
  set_filename = folder + '.pickle'
  test_datasets.append(set_filename)

In [4]:
train_dataset = []
num=0
for file in train_datasets:
  f = open(file,"rb")
  dataset = pickle.load(f)
  num = num +1
  train_dataset.append(dataset[0:10000,:,:])
  print(train_dataset[num-1].shape)

(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)
(10000, 28, 28)


In [5]:
test_dataset = []
num = 0
for file in test_datasets:
  f = open(file,"rb")
  dataset = pickle.load(f)
  num = num +1
  test_dataset.append(dataset)
  print(dataset.shape)


## split train dataset to get validation set
train_labels = []
test_labels = []
for label in range(len(train_datasets)):
  for i in range(10000):
    train_labels.append(label) 
    
for label in range(len(test_datasets)):
  for i in range(test_dataset[label].shape[0]):
    test_labels.append(label) 
    

(1872, 28, 28)
(1873, 28, 28)
(1873, 28, 28)
(1873, 28, 28)
(1873, 28, 28)
(1872, 28, 28)
(1872, 28, 28)
(1872, 28, 28)
(1872, 28, 28)
(1872, 28, 28)


In [6]:
temp = []
for i in range(10):
  for j in range(train_dataset[i].shape[0]):
    temp.append(train_dataset[i][j])

train_dataset, temp = temp, train_dataset

temp = []
for i in range(10):
  for j in range(test_dataset[i].shape[0]):
    temp.append(test_dataset[i][j])

test_dataset, temp = temp, test_dataset

test_dataset = np.array(test_dataset)
test_dataset.shape

(18724, 28, 28)

In [8]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
train_dataset = np.array(train_dataset)
test_dataset = np.array(test_dataset)

In [10]:
train_dataset = train_dataset.reshape((-1,784))
test_dataset = test_dataset.reshape((-1,784))

In [11]:
train_dataset, valid_dataset, train_labels, valid_labels = train_test_split(train_dataset, train_labels, test_size = 0.1)

In [13]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  labels = labels.reshape((-1,10))
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (90000, 28, 28, 1) (90000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (18724, 28, 28, 1) (18724, 10)


In [14]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [15]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [16]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 1.224264
Minibatch accuracy: 0.0%
Validation accuracy: 90.2%
Minibatch loss at step 50: nan
Minibatch accuracy: 93.8%
Validation accuracy: 90.2%
Minibatch loss at step 100: nan
Minibatch accuracy: 93.8%
Validation accuracy: 90.2%
Minibatch loss at step 150: nan
Minibatch accuracy: 62.5%
Validation accuracy: 90.2%
Minibatch loss at step 200: nan
Minibatch accuracy: 100.0%
Validation accuracy: 90.2%
Minibatch loss at step 250: nan
Minibatch accuracy: 93.8%
Validation accuracy: 90.2%
Minibatch loss at step 300: nan
Minibatch accuracy: 87.5%
Validation accuracy: 90.2%
Minibatch loss at step 350: nan
Minibatch accuracy: 81.2%
Validation accuracy: 90.2%
Minibatch loss at step 400: nan
Minibatch accuracy: 100.0%
Validation accuracy: 90.2%
Minibatch loss at step 450: nan
Minibatch accuracy: 100.0%
Validation accuracy: 90.2%
Minibatch loss at step 500: nan
Minibatch accuracy: 87.5%
Validation accuracy: 90.2%
Minibatch loss at step 550: nan
Minibatch accurac

In [17]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    shape = pool.get_shape().as_list()
    print(shape)
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 7, 7, 16]
[10000, 7, 7, 16]
[18724, 7, 7, 16]


In [ ]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 0.167398
Minibatch accuracy: 100.0%
